# Laboratorium 5 - rekomendacje grafowe

## Przygotowanie

 * dataset i potrzebne biblioteki są dokładnie takie same jak na poprzednim laboratorium
 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn gensim==3.8.3`

## Część 1. - przygotowanie danych

In [1]:
import math
import random
import numpy as np
import pandas

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold

C:\Users\Mateusz\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
SCORE_THRESHOLD = 4.0 # recenzje z co najmniej taka ocena wezmiemy pod uwage
VECTOR_SIZE = 20 # jak dlugie powinny byc wektory osadzen wierzcholkow
NEIGHBOURS_WINDOW = 11 # tylu sasiadow wezmiemy pod uwage w algorytmie Word2Vec (symetrycznie i wliczajac biezacy element)
PATH_LENGTH = 30 # dlugosc pojedynczej losowej sciezki
PATHS_COUNT_PER_NODE = 20 # liczba losowych sciezek zaczynajacych sie w kazdym z wierzcholkow

In [3]:
# wczytujemy oceny uytkownikow

ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
ratings = ratings.where(ratings['rating'] >= SCORE_THRESHOLD).dropna()
# rozszerzamy ID tak, by sie nie powtarzaly
ratings['userId'] = ratings['userId'].apply(lambda x: 'u_' + str(int(x)))
ratings['movieId'] = ratings['movieId'].apply(lambda x: 'm_' + str(int(x)))
ratings

,userId,movieId,rating
0,u_1,m_1,4.0
1,u_1,m_3,4.0
2,u_1,m_6,4.0
3,u_1,m_47,5.0
4,u_1,m_50,5.0
...,...,...,...
100830,u_610,m_166528,4.0
100831,u_610,m_166534,4.0
100832,u_610,m_168248,5.0
100833,u_610,m_168250,5.0


In [4]:
# wczytujemy gatunki filmow

movies = pandas.read_csv('ml-latest-small/movies.csv').drop(columns=['title'])
movies['movieId'] = movies['movieId'].apply(lambda x: 'm_' + str(int(x)))
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
movies_to_genres = movies.explode('genres')
movies_to_genres['genres'] = movies_to_genres['genres'].apply(lambda x: 'g_' + x.lower())
movies_to_genres = movies_to_genres.rename(columns = {'genres': 'genre'})
movies_to_genres

,movieId,genre
0,m_1,g_adventure
0,m_1,g_animation
0,m_1,g_children
0,m_1,g_comedy
0,m_1,g_fantasy
...,...,...
9738,m_193583,g_fantasy
9739,m_193585,g_drama
9740,m_193587,g_action
9740,m_193587,g_animation


In [5]:
users = ratings['userId'].unique()
movies = movies_to_genres['movieId'].unique()
genres = movies_to_genres['genre'].unique()

## Część 2. - spacer po grafie

In [6]:
# generujemy losowe sciezki w grafie
#   krawedzie reprezentowane sa w dwoch macierzach - ratings i movies
#   w wersji podstawowej wszystkie krawedzie traktujemy jako niewazone i nieskierowane
#   mozliwe ulepszenia:
#    - rozwazenie krawedzi skierowanych
#    - uwzglednienie wag krawedzi (ocen uzytkownikow)
#    - jakas forma normalizacji - obnizenia wag wierzcholkow o wysokich stopniach
#    - Node2Vec - parametry P i Q
# wynikiem powinna byc lista list - kazda z tych list zawiera kolejne ID wierzcholkow na sciezce

def generate_walks(ratings, movies_to_genres, paths_per_node, path_length):
    paths = []
    nodes = np.concatenate((users, movies, genres))
    graph = {n : [] for n in nodes}
    for rating in ratings.to_numpy():
        graph[rating[0]].append(rating[1])
        graph[rating[1]].append(rating[0])
    for movie in movies_to_genres.to_numpy():
        graph[movie[0]].append(movie[1])
        graph[movie[1]].append(movie[0])
 
    for node in nodes:
        for _ in range(paths_per_node):
            current_node = node
            path = [current_node]
            for _ in range(path_length-1):
                idx = random.randrange(0, len(graph[current_node]))
                next_node = graph[current_node][idx]
                path.append(next_node)
                current_node = next_node
            paths.append(path)
    return paths, graph
    
walks, graph = generate_walks(ratings, movies_to_genres, PATHS_COUNT_PER_NODE, PATH_LENGTH)

## Część 3. - obliczenie osadzeń

In [7]:
# trenujemy model
#   zauwaz, ze wszystkie trzy rodzaje wierzcholkow beda reprezentowane tak samo, w tej samej przestrzeni

model = Word2Vec(sentences=walks, vector_size=VECTOR_SIZE, window=5, min_count=1, workers=4)
embeddings = model.wv

## Część 4. - rekomendacje i zastosowania

In [8]:
PULP_FICTION = 'm_296'
TOY_STORY = 'm_1'
PLANET_OF_THE_APES = 'm_2529'

In [9]:
# wyszukajmy K najpodobniejszych filmów do danego
# porownaj wyniki dla odleglosci euklidesowej i cosinuswej, np. na trzech powyzszych filmach
from math import sqrt

def euclidian_distance(i, j):
    return sqrt(np.sum(np.square(np.sum(embeddings[i] - embeddings[j])))) 

def cosine_distance(i, j):
    return np.sum(np.multiply(embeddings[i], embeddings[j])) / sqrt(np.sum(np.square(embeddings[i]))) * sqrt(np.sum(np.square(embeddings[j])))

def k_most_similar_movies(movie_id, K, embeddings, distance_fun):
    if distance_fun is cosine_distance:
        return sorted(movies, key= lambda movie: abs(distance_fun(movie_id, movie)), reverse=True)[1:K+1] #skip movie_id
    return sorted(movies, key= lambda movie: abs(distance_fun(movie_id, movie)))[1:K+1] #skip movie_id

k_most_similar_movies(PULP_FICTION, 5, embeddings, euclidian_distance)
k_most_similar_movies(TOY_STORY, 5, embeddings, cosine_distance)

['m_296', 'm_356', 'm_527', 'm_593', 'm_110']

In [10]:
# wyszukajmy k filmow najblizszych uzytkownikowi
# wykorzystaj funkcje z poprzedniej komorki

def k_best_movies_for_user(user_id, K, embeddings, distance_fun):
    user_movies = graph[user_id]
    movies_to_recomend = [movie for movie in movies if movie not in user_movies]
    if distance_fun is cosine_distance:
        return sorted(movies_to_recomend, key= lambda movie: abs(distance_fun(user_id, movie)), reverse=True)[:K] #skip movie_id
    return sorted(movies_to_recomend, key= lambda movie: abs(distance_fun(user_id, movie)))[:K] #skip movie_id

k_best_movies_for_user('u_1', 10, embeddings, cosine_distance)

['m_88163',
 'm_46578',
 'm_56367',
 'm_56782',
 'm_2324',
 'm_92259',
 'm_8533',
 'm_4973',
 'm_63082',
 'm_7323']

In [11]:
# sprobujmy czegos bardziej skomplikowanego
#   znajdz ulubiony gatunek filmowy uzytkownika
#   a nastepnie zaproponuj K filmow z tego gatunku - ale nie tych najblizszych uzytkownikowi
#   (zaproponuj, w jaki sposob dobrac filmy interesujace, ale nie z najblizszego otoczenia)
from collections import defaultdict

def k_from_favourite_genre(user_id, K, embeddings, distance_fun):
    user_movies_to_recomend = k_best_movies_for_user(user_id, K, embeddings, distance_fun)
    user_movies = graph[user_id]
    liked_movies = ratings.where(ratings['userId'] == user_id).where(ratings['rating'] >= SCORE_THRESHOLD).dropna().to_numpy()[:,1]

    movies_to_recomend = [movie for movie in movies if movie not in user_movies_to_recomend and movie not in user_movies]
    fav_genres = defaultdict(int) #most popular
    for liked_movie in liked_movies:
        genres = movies_to_genres.where(movies_to_genres['movieId'] == liked_movie).dropna().to_numpy()[:,1]
        for genre in genres:
            fav_genres[genre] += 1
            
    favourite_genre_id = max(fav_genres, key=fav_genres.get)
        
    if distance_fun is cosine_distance:
        return sorted(movies_to_recomend, key= lambda movie: abs(distance_fun(favourite_genre_id, movie)), reverse=True)[:K] #skip movie_id
    return sorted(movies_to_recomend, key= lambda movie: abs(distance_fun(favourite_genre_id, movie)))[:K] #skip movie_id

k_from_favourite_genre('u_1', 10, embeddings, cosine_distance)

['m_296',
 'm_318',
 'm_7361',
 'm_4226',
 'm_4995',
 'm_1246',
 'm_1035',
 'm_3949',
 'm_1704',
 'm_1193']

In [12]:
# Na koniec najbardziej skomplikowany algorytm - odpowiednik "Radia utworu" w Spotify.
#   Zaczynamy od jednego filmu, a nastepnie wyznaczamy kolejne, wedrujac po przestrzeni, w ktorej wszystkie elementy sa osadzone.
#   Zaproponuj, jak zdefiniowac podzbior filmow, z ktorych bedziemy wybierac (np. filmy odlegle o min. a i max. b od danego)
#   oraz jak generowac kolejny skok (tak, zeby seria rekomendacji nie byla zbyt monotonna, ale rownoczesnie zgodna z gustem uzytkownika)


def get_playlist(start_movie_id, user_id, K, embeddings):
    min_threshold = 0.15
    max_threshold = 0.5
    reduced_movies_list = sorted(movies, key= lambda movie: abs(cosine_distance(user_id, movie)))[int(min_threshold*len(movies)):int(max_threshold*len(movies))]
    playlist = [start_movie_id]
    for i in range(K):
        new_recommendation = sorted(reduced_movies_list, key= lambda movie: abs(cosine_distance(playlist[-1], movie)))[0]
        reduced_movies_list.remove(new_recommendation)
        playlist.append(new_recommendation)
    return playlist[1:]

get_playlist('m_1', 'u_1', 20, embeddings)

['m_7647',
 'm_5612',
 'm_38798',
 'm_4212',
 'm_2261',
 'm_95182',
 'm_49280',
 'm_8934',
 'm_3723',
 'm_42725',
 'm_113280',
 'm_27879',
 'm_2136',
 'm_8191',
 'm_97836',
 'm_8979',
 'm_146309',
 'm_4835',
 'm_6764',
 'm_645']